In [1]:
import numpy as np
import pandas as pd
import os
import time
import sys

#bla bla

import sklearn
import psutil
import pickle

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from copy import deepcopy as dc


### Dati

In [2]:
with open('..\\dati\\step_1_dati_out.pickle', 'rb') as f:
    df = pickle.load(file=f)
y = df["survived"]
df = df.drop(columns = "survived")
df.head(3)

,age,fare,embarked,sex,pclass
0,29.0000,211.3375,S,female,1.0
1,0.9167,151.5500,S,male,1.0
2,2.0000,151.5500,S,female,1.0


In [3]:
step_2 = {}

### Scaler

In [4]:
numeric_features = ["age", "fare"]
stand_scaler = ColumnTransformer(
    transformers=[("scaler", StandardScaler(), numeric_features)])

fit_scaler = stand_scaler.fit(df)
fitted_scaler = dc(fit_scaler)

step_2["step_2_scaler_num"] = {"obj":fitted_scaler,
                           "fitted_on":numeric_features}
df.loc[::,numeric_features] = fit_scaler.transform(df)
df.shape

(1309, 5)

### OneHotEncoding


In [5]:
categorical_features = ["embarked", "sex", "pclass"]
ohe = ColumnTransformer(
    transformers=[("ohe", OneHotEncoder(handle_unknown="error"),categorical_features)])

fit_ohe = ohe.fit(df)
fitted_ohe = dc(fit_ohe)

step_2["step_2_ohe"] = {"obj":fitted_ohe,
                        "fitted_on":categorical_features}
df = pd.concat([df, pd.DataFrame(fit_ohe.transform(df), columns = fit_ohe.get_feature_names_out())],axis=1).drop(columns=categorical_features)
df.shape

(1309, 10)

### save pickle

In [6]:
save_path = os.getenv('PATH_NAME',
        'C:\\Users\\AndreaNicoletta\\Desktop\\Elyra\\pipelines\\titanic\\dati\\')
with open(save_path+'step_2_dict_out.pickle', 'wb') as handle:
    pickle.dump(step_2, handle, protocol=pickle.HIGHEST_PROTOCOL)
df["survived"] = y
with open(save_path+'step_2_dati_out.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)